**ENTRENAMIENTO DE TERSONFLOW PARA DATA DE ELECTROCARDIOGRAMAS**

#NOTACION A USAR *son las variables q no son pertinentes en el estudio
NOTA: Los datos originales poseen una fila que se debe eliminar a mano ya que la misma tiene mas columnas que las otras
1. survival -- the number of months patient survived (has survived, if patient is still alive).  
2. still-alive -- a binary variable.  0=dead at end of survival period, 1 means still alive 
3. age-at-heart-attack -- age in years when heart attack occurred
4. pericardial-effusion -- binary. Pericardial effusion is fluid around the heart.  0=no fluid, 1=fluid
5. fractional-shortening -- a measure of contracility around the heart lower numbers are increasingly abnormal
6. epss -- E-point septal separation, another measure of contractility. Larger numbers are increasingly abnormal.
7. lvdd -- left ventricular end-diastolic dimension.  This is a measure of the size of the heart at end-diastole. Large hearts tend to be sick hearts.
8. wall-motion-score -- a measure of how the segments of the left ventricle are moving
9. wall-motion-index -- equals wall-motion-score divided by number of segments seen.  Usually 12-13 segments are seen in an echocardiogram.  Use this variable INSTEAD of the wall motion score.
10. mult -- a derivate var which can be ignored
11. name -- the name of the patient (I have replaced them with "name")
12. group -- meaningless, ignore it
13. alive-at-1 -- Boolean-valued. Derived from the first two attributes. 0 means patient was either dead after 1 year or had been followed for less than 1 year.  1 means patient was alive at 1 year.

**PAQUETERIAS NECESARIAS**

In [1]:
import tensorflow as tf
import random
from tensorflow import keras
# Bibliotecas de ayuda
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
#Biblioteca para datos de entrenamiento
from sklearn.model_selection import train_test_split

**ENTRADA DE LA INFORMACION, ORDENAMIENTO DE LA INFORMACION A USAR**

In [2]:
# NOTACION DE LAS VARIABLES 
label_total=["survival", "still_alive", "age-at-heart-attack",
       "pericardial-effusion","fractional-shortening", 
       "epss", "lvdd","wall-motion-score","wall-motion-index",
       "mult" , "group", "alive-at-1"];
# ENTRAR LA INFORMACION DESDE UN ARCHIVO INTERNO
data = pd.DataFrame(np.genfromtxt('echocardiogram.data', delimiter=','), columns= label_total)

**FILTRADO DE LAS VARIABLES QUE SE UTILIZARAN**

In [3]:
label_user=["age-at-heart-attack", "pericardial-effusion","fractional-shortening", "epss",
            "lvdd","wall-motion-index", "alive-at-1"]; # ELECCION DE LAS COLUMNAS
select_data=data[label_user];#DATOS ORIGINALES SIN FILTRAR, COLUMNAS DE INTERES

**CONOCER EL OUTPUT**

Este se relaciona con la posibilidad (en forma binaria) de encontrar al paciente vivo o muerto al año del infarto. El formato es de dos columnas y las opciones permitidas son [1,0],[0,1],[-1,-1]

In [4]:
#print(select_data["alive-at-1-false"])

In [5]:
select_data["alive-at-1-false"]=0;  #CREAR COLUMNA EXTRA
for j in range(len(select_data)):   #SELECCION DE CONDICIONES
    if   data[label_total[0]].values[j] > 12:   #VIVIO MAS DE 12 MESES
         select_data["alive-at-1"].values[j] = 1;       #CUMPLE EL REQUISITO
         select_data["alive-at-1-false"].values[j] = 0; 
    elif data[label_total[0]].values[j] == 12 and data[label_total[1]].values[j] == 1:  #CASO PARTICULAR, 12 MESES Y VIVE
         select_data["alive-at-1"].values[j] = 1;       #CUMPLE EL REQUISITO
         select_data["alive-at-1-false"].values[j] = 0; 
    elif data[label_total[0]].values[j] < 12 and data[label_total[1]].values[j] == 1:  #MENOS DE 12 MESES Y VIVE
         select_data["alive-at-1"].values[j] = -1;      #NO CUMPLE EL REQUISITO
         select_data["alive-at-1-false"].values[j] = -1;    
    else:   # EL RESTO DE LOS DATOS
         select_data["alive-at-1"].values[j] = 0;       #CUMPLE EL REQUISITO DE ESTAR MUERTO
         select_data["alive-at-1-false"].values[j] = 1;
#    if  random.random()>.5:
#        select_data["alive-at-1"].values[j] = 0; #ruido
#        select_data["alive-at-1-false"].values[j] = 1;

C:\Users\Dell\.conda\envs\ML\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [6]:
#select_data["alive-at-1-"]=-1
#print(select_data)

**MATRIX LOGICA DE DATOS ELIMINAR LAS FILAS CON -1 (DESCONOCIDO)**

In [7]:
for i in range(len(label_user)):
    if i==0:
        log=select_data[label_user[i]] !=-1;#print(i);
    else:
        log= np.logical_and(select_data[label_user[i]] !=-1,log)
select_data=select_data[log]; # filtrar los desconocidos           

**DEFINICION DE MIS VALORES INPUT Y OUTPUT**

In [8]:
input_data, input_data_test, output_data, output_data_test = train_test_split(
    select_data[{"age-at-heart-attack", "pericardial-effusion","fractional-shortening", 
                 "epss", "lvdd","wall-motion-index"}], 
    select_data[{"alive-at-1","alive-at-1-false"}], test_size=0.2, shuffle = True);
    #select_data[{"alive-at-1"}], test_size=0.2, shuffle = True);

**PARAMETROS INICIALES DE ENTRENAMIENTO**

In [9]:
n_input = np.int64(len(label_user)-1)   # input layer
n_hidden1 = np.round(np.int64(len(label_user)/2))#np.int64(np.round(np.sqrt(len(label_user)))) # 1st hidden layer
n_hidden2 = 256 # 2nd hidden layer
n_output = np.int64(2)   # output 

learning_rate = 1e-4;   n_iterations = 1000   ;   batch_size = 3;         dropout = 0.5

X = tf.compat.v1.placeholder("float", [None, n_input]); Y = tf.compat.v1.placeholder("float", [None, n_output])

weights = {'w1': tf.Variable(tf.random.normal([n_input, n_hidden])),
    'out': tf.Variable(tf.random.normal([n_hidden, n_output]))}

biases = {'b1': tf.Variable(tf.constant(0.1, shape=[n_hidden])),
    'out': tf.Variable(tf.constant(0.1, shape=[n_output]))}

layer_1 = tf.add(tf.matmul(X, weights['w1']), biases['b1'])
output_layer = tf.matmul(layer_1, weights['out']) + biases['out']

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=output_layer))
train_step = tf.compat.v1.train.AdamOptimizer(learning_rate).minimize(cross_entropy)

correct_pred = tf.equal(tf.argmax(output_layer, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

**INICIALIZAR EL ENTRENAMIENTO**

In [10]:
init = tf.compat.v1.global_variables_initializer()

cost_summary = tf.compat.v1.summary.scalar("Cost", cross_entropy)
acc_summary = tf.compat.v1.summary.scalar("Accuracy", accuracy)
all_summary = tf.compat.v1.summary.merge_all()

In [11]:
#print(input_data.values,output_data.values)

**ENTRENAMIENTO**

In [12]:
#saver=tf.train.Saver() #SALVAR LA SECCION PARA PODERLA UTILIZAR LUEGO
with tf.Session() as sess:
    writer = tf.summary.FileWriter("Tensorboard", sess.graph)
    #print(sess)
    sess.run(init)
    #saver.save(sess, 'modelo')
    for i in range(n_iterations):
        sess.run(train_step, feed_dict={X: input_data.values, Y: output_data.values})
        
        summary_results, loss, acc = sess.run([all_summary, cross_entropy, accuracy], feed_dict={X: input_data.values, Y: output_data.values})
        writer.add_summary(summary_results, i)
        if i%200 == 0:
            print("Iteration", str(i), "\t| Loss =", str(loss), "\t| Accuracy =", str(acc))
    
        if acc==1:
            break
    test_accuracy = sess.run(accuracy, feed_dict={X: input_data_test.values, Y: output_data_test.values})
    print("\nAccuracy on test set:", test_accuracy)
    
    a, b = sess.run([tf.argmax(output_layer, 1),tf.argmax(Y,1)], feed_dict={X: input_data.values, Y: output_data.values}) 
    
    for ea, eb in zip(a, b):
        print(ea,eb)
    #saver.save(sess, 'modelo') #SALVAR EL MODELO EXTERNAMENTE

Iteration 0 	| Loss = 4.9490314 	| Accuracy = 0.84615386
Iteration 200 	| Loss = 4.696648 	| Accuracy = 0.84615386
Iteration 400 	| Loss = 4.4564056 	| Accuracy = 0.84615386
Iteration 600 	| Loss = 4.2276564 	| Accuracy = 0.84615386
Iteration 800 	| Loss = 4.009779 	| Accuracy = 0.84615386

Accuracy on test set: 0.8235294
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 0
1 1
1 1
1 1
1 1
1 1
1 1
0 1
1 1
1 1
0 1
1 1
1 1
1 1
1 1
0 1
1 0
0 1
1 1
1 1
1 1
1 1
1 0
1 1
1 1
1 1
0 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 0
1 0
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1
1 1


**PROBAR EL MODELO CON LA DATA DE MUESTRA**

In [13]:
print(input_data.size)

390
